### Output files with model FRP vs ks parameter data for weak (vs=1.0) and strong (vs=1.6) oscillators

In [ ]:
"""
Model of circadian oscillator with entrainment
"""
import numpy as np
import matplotlib as plt
import pandas as pd
from scipy.integrate import odeint
from pylab import *
from scipy import signal
from scipy.signal import argrelextrema
%matplotlib inline

#---------------------------------------------------------------------------------------------
#    Increasing hill function
#---------------------------------------------------------------------------------------------    
def hill_up(x, k, n):
    h = np.power(x,n)/(np.power(k,n) + np.power(x,n))
    return h

#---------------------------------------------------------------------------------------------
#    Decreasing hill function
#---------------------------------------------------------------------------------------------    
def hill_down(x, k, n):
    h = np.power(k,n)/(np.power(k,n) + np.power(x,n))
    return h

#---------------------------------------------------------------------------------------------
#    Generage a squate function of time series t with given period and amplitude
#---------------------------------------------------------------------------------------------
def square_wave(t, period, amp):
    wave = amp*signal.square(2 * np.pi * t/period)+amp
    return wave
            
#---------------------------------------------------------------------------------------------
#    Function defining differential equation for the model
#---------------------------------------------------------------------------------------------
def df(y, t):
    #assign input vector,y to variables of the model
    M  = y[0]
    PC = y[1]
    PN = y[2]

 
    # mRNA expression
    dM =  vs*hill_down(PN, KI, 4) - vm*hill_up(M, Km, 1) - square_wave(t, light_period, light_amp)*M
    # Protein concentration, PC - celllular, PN - nuclear
    dPC =   ks*M + k2*PN - k1*PC    - vd*hill_up(PC, Kd, 1)
    dPN =        - k2*PN + k1*PC
    # output right-hand side of differetial equations
    return [dM, dPC, dPN]

#---------------------------------------------------------------------------------------------
#    Find location and amplitude of peaks in a variable v with given timeline t
#---------------------------------------------------------------------------------------------
def find_peaks(t,v):
    # convert vontage trace into array
    v_array = np.array(v, dtype=np.float)
    # Find indices of local max(spikes)
    locs_max = argrelextrema(v_array, np.greater, order=2)[0]
    # Record spike times and max voltages in a dataframe
    peak_times = t[locs_max]
    peak_amp   = v[locs_max]
    return(peak_times, peak_amp)

#---------------------------------------------------------------------------------------------
#    Plot all variable of the model for the last plotting_time hours
#    Each plot is labeled by information in list names
#---------------------------------------------------------------------------------------------
def periods_list(soln,dt, names, non_transient_time):
    t  = np.arange(0,non_transient_time, dt)
    n_ind = len(t)
    n_vars = len(variable_names)
    periods = []
    for i in range(0,n_vars):
        variable = soln[-n_ind:,i]
        [peaks, amp] = find_peaks(t,variable)
        periods.append(peaks[-1]-peaks[-2])
    return periods

#---------------------------------------------------------------------------------------------
#    Parameters of the model
#---------------------------------------------------------------------------------------------
vs = 1.6; vm = 0.505; Km = 0.5
KI = 1.0; ks = 0.2; vd = 1.4
Kd = 0.13; k1 = 0.5; k2 = 0.6

#---------------------------------------------------------------------------------------------
# Parameters for light stimulus (amplitude 0 means DD)
#---------------------------------------------------------------------------------------------
light_period = 24
light_amp = 0

#---------------------------------------------------------------------------------------------
#   Initial conditions
#---------------------------------------------------------------------------------------------
y0= [2.15404051, 3.19312247, 3.11948775] 

# List with variable names
variable_names = ['mRNA', 'Cytosol protein', 'Nuclear Protein']

#---------------------------------------------------------------------------------------------
#   Run the simulation of the model 
#---------------------------------------------------------------------------------------------
# Choose total time of simulation, T
T = 1000.
# Chose time step
dt=0.05
# Make a time grid for the simulation
nsteps=int(T/dt)
t  = np.linspace(0, T, nsteps) 
# Chose number of simulation hours(last) which will be the data portion
non_transient_T = 100
vs_list = [1.0,1.6]
ks_list = [0.11 + 0.01*i for i in range(0,90)]
#ks_list = [0.11 + 0.1*i for i in range(0,10)]
for vs in vs_list:
    period_list = []
    print(ks_list)
    for ks in ks_list:
        # Solve ODE for the model and plot the solution
        soln = odeint(df, y0, t)
        periods_ret = periods_list(soln, dt, variable_names, non_transient_T)
        period_avg = np.array(periods_ret).mean()
        print('vs = ',vs,' ks = ',ks,': period_avg = ',period_avg)
        period_list.append(period_avg)
    
    dfout = pd.DataFrame(list(zip(ks_list, period_list)), columns =['ks', 'period'])
    filename = 'ks_vs_periods_vs=' + str(vs) + '.dat'
    dfout.to_csv(filename,index=False)